**HateXplain Dataset**

Three categories: NEUTRAL, HATESPEECH, OFFENSIVE

Since it is used as a hatespeech dataset and offensive language usually does not consitute hatespeech, here only the hatespeech category was used in the binary classification as hatespeech.
(Though this might be an open question for debate comparing it to the other datasets).

There are alway three annotator votes given. Decided label based on Majority vote. For Multilabel they can be UNDECIDED.

LABEL MUTLICLASS ('category'):
0 - NEUTRAL
1 - HATESPEECH
2 - OFFENSIVE
3 - UNDECIDED

LABEL
0 - NEUTRAL
1 - HATESPEECH



In [1]:
import os
import sys
import pandas as pd
from prep_collection import PrepCollection as prep
import json
import numpy as np
from collections import Counter

In [2]:
wdr_path = os.path.dirname(os.path.dirname(os.getcwd()))
ds_raw_path = os.path.join(wdr_path + "/Datasets/Hate Speech/HateXplain/Data/dataset.json")

In [3]:
def preprocess_hatexplain(wdr_path, ds_raw_path):
    with open(ds_raw_path, 'r') as f:
        ds_json = json.load(f)

    for key in ds_json.keys():
        annotators = ds_json[key]['annotators']
        slst = [ele['label'] for ele in annotators]
        words_count = Counter(slst)
        if words_count['normal'] >= 2:
            label = 0
            category = 0 # NEUTRAL
        elif words_count['offensive'] >= 2:
            label = 0
            category = 2 # OFFENSIVE
        elif words_count['hatespeech'] >= 2:
            label = 1
            category = 1 # HATESPEECH
        else:
            label = 0 # put it into category null since I didn't count offensive as hatespeech
            category = 3 # UNDECIDED
        ds_json[key]['label'] = label
        ds_json[key]['category'] = category
        ds_json[key]['text'] = " ".join(ds_json[key]['post_tokens'])

    df_hatexplain = pd.DataFrame(ds_json).transpose()
    df = df_hatexplain.loc[:,['text', 'label', 'category']]
    df['text'] = df['text'].apply(prep.prepare_text)
    df['id'] = range(len(df))
    df.to_csv(os.path.join(wdr_path + "/Preprocessed_Datasets/092-HateXplain.csv"))

In [4]:
preprocess_hatexplain(wdr_path, ds_raw_path)